In [1]:
%reload_ext autoreload
%autoreload 2

In [4]:
import configparser, networkx as nx, itertools, math, json, sys, time, timeit, numpy, pdb, jsbeautifier, re, spacy
from tqdm import tqdm
from pathlib import Path

### Grounding
Extract and match concepts from Ai2Thor objects list to ConceptNet concepts.

In [2]:
from grounding.grounding_concepts import GroundConcepts

In [15]:
config_path = Path("/Users/odoemoo1/projects/visual-navigation/KagNet/grounding/paths.cfg")
ground = GroundConcepts(config_path)

In [ ]:
ground.process('/Users/odoemoo1/projects/visual-navigation/KagNet/grounding/objects.txt', )

Added rooms


grounding:  41%|████▏     | 36/87 [03:58<05:42,  6.72s/it]

In [14]:
%debug

> /Users/odoemoo1/projects/visual-navigation/KagNet/grounding/grounding_concepts.py(46)load_matcher()
     44 
     45     def load_matcher(self,):
---> 46         with open(self.path / self.config["paths"]["matcher_patterns"][3:], "r", encoding="utf8") as f:
     47             all_patterns = json.load(f)
     48 

ipdb> self.path
PosixPath('/Users/odoemoo1/projects/visual-navigation/KagNet')
ipdb> self.config["paths"]["matcher_patterns"][3:]
'matcher_patterns.json'
ipdb> q


In [41]:
from grounding.grounding_concepts import match_mentioned_concepts, camel_case_split

grounding_config_path = Path("/Users/odoemoo1/projects/visual-navigation/KagNet/grounding/paths.cfg")

opts = jsbeautifier.default_options()
opts.indent_size = 2

def ai2thor_objects_to_objects_concepts_all(add_rooms=False, exclude_self=True):
    nlp = spacy.load('en', disable=['ner', 'parser', 'textcat'])
    nlp.add_pipe(nlp.create_pipe('sentencizer'))

    with open('./grounding/objects.txt', 'r') as f: 
        lines = f.read().split('\n')
   
    objects = [line for line in lines if not line == '']
    objects = [' '.join(np.unique([o] + camel_case_split(o))) for o in objects]
    objects_all = objects
    name = ''
    
    if add_rooms:
        objects_all += ['Kitchen', 'Bedroom', 'Bathroom', 'LivingRoom']
        name += '_rooms'
        
        print('Added rooms')
    if exclude_self:
        _objects_all = [' '.join(objects_all[:ii] + objects_all[ii+1:]) for ii, oo in enumerate(objects)]
        name += '_exclude_self'
    else:
        _objects_all = [' '.join(objects_all) for ii, oo in enumerate(objects)]
        name += '_include_self'
        
    res = match_mentioned_concepts(nlp, sents=objects, answers=_objects_all, config_path=grounding_config_path)
    
    fn = f'./datasets/ai2thor_objects_to_objects{name}_concepts.json'
    with open(fn, 'w') as f: 
        json.dump(res, f)
    
    with open(fn + '.mcp', 'w') as fp:
        fp.write(jsbeautifier.beautify(json.dumps(res), opts))
    
    print(f'\nSaved to {fn}')


In [42]:
ai2thor_objects_to_objects_concepts_all(exclude_self=False)

grounding batch_id:-1: 100%|██████████| 83/83 [08:11<00:00,  5.93s/it]



Saved to ./datasets/ai2thor_objects_to_objects_include_self_concepts.json


In [43]:
ai2thor_objects_to_objects_concepts_all(add_rooms=True, exclude_self=False)

Added rooms


grounding batch_id:-1: 100%|██████████| 87/87 [08:56<00:00,  6.17s/it]



Saved to ./datasets/ai2thor_objects_to_objects_rooms_include_self_concepts.json


We can skip pruning step as we're not dealing with sentences. 

### Generate Conceptnet graph

Run below in cell:
```
!cd ./pathfinder/`
!python graph_construction.py`
!cd ..
```

Outputs networkX graph of conceptnet.

### Path finding
Find a path between the concepts (i.e. objects).

Re-factored pathfinder.py to create an object. 

In [7]:
from pathfinder.pathfinder import PathFinder

In [10]:
pathfinder = PathFinder( '/Users/odoemoo1/projects/visual-navigation/KagNet/pathfinder/paths.cfg')

In [16]:
fn = './datasets/ai2thor/ai2thor_objects_to_objects_rooms_include_self_concepts.json'
with open(fn, 'r') as fp:
    mcp_data = json.load(fp)

nhops = 4
pf = []
for item in tqdm(mcp_data):
    acs = sorted(item["ac"])
    qcs = item["qc"]
    pfr_qa = []  # path finding results
    
    for ac in acs:
        for qc in qcs:
            pf_res = pathfinder.find_paths(qc, ac, nhops, ifprint=True)
            pfr_qa.append({"ac":ac, "qc":qc, "pf_res":pf_res})
        break
    pf.append(pfr_qa)



  1%|          | 1/87 [00:00<00:33,  2.60it/s]

clock ----[relatedto]---> alarm
clock ----[relatedto*]---> bed ----[relatedto]---> alarm
clock ----[relatedto*]---> bedroom ----[relatedto]---> alarm
clock ----[relatedto]---> arms ----[relatedto*]---> alarm
clock ----[relatedto]---> signal ----[isa*]---> alarm
clock ----[relatedto*/usedfor*/isa*]---> alarm_clock ----[relatedto*/relatedto]---> alarm
clock ----[isa/relatedto]---> device ----[relatedto*/isa*]---> alarm
clock ----[isa]---> non_agentive_artifact ----[isa*]---> alarm
clock ----[atlocation]---> house ----[atlocation*]---> fire ----[relatedto]---> alarm
clock ----[atlocation]---> house ----[relatedto*/atlocation*]---> bed ----[relatedto]---> alarm
clock ----[atlocation]---> house ----[relatedto*/atlocation/atlocation*/relatedto]---> bedroom ----[relatedto]---> alarm
clock ----[atlocation]---> house ----[relatedto*]---> bell ----[relatedto]---> alarm
clock ----[relatedto]---> around ----[relatedto*]---> ring ----[relatedto*/usedfor*]---> alarm
clock ----[relatedto]---> tell --

  2%|▏         | 2/87 [00:00<00:36,  2.36it/s]

apple ----[relatedto]---> basic ----[relatedto*]---> sound ----[relatedto]---> alarm
apple ----[relatedto]---> lunch ----[relatedto*]---> morning ----[relatedto]---> alarm
apple ----[relatedto]---> sometimes ----[relatedto*]---> bed ----[relatedto]---> alarm
apple ----[relatedto]---> sometimes ----[relatedto*]---> ring ----[relatedto*/usedfor*]---> alarm
apple ----[relatedto]---> sometimes ----[relatedto*]---> shock ----[relatedto]---> alarm
apple ----[relatedto]---> sometimes ----[relatedto*]---> surprise ----[relatedto*]---> alarm
apple ----[relatedto*/hasproperty/relatedto]---> red ----[relatedto*]---> fire ----[relatedto]---> alarm
apple ----[relatedto*/hasproperty/relatedto]---> red ----[relatedto*]---> bed ----[relatedto]---> alarm
apple ----[relatedto*/hasproperty/relatedto]---> red ----[relatedto*/relatedto]---> danger ----[relatedto*/causes*/relatedto]---> alarm
apple ----[relatedto]---> digital ----[relatedto*]---> clock ----[relatedto]---> alarm
apple ----[isa]---> plant ---

slice ----[relatedto]---> action ----[relatedto]---> device ----[relatedto*/isa*]---> alarm
slice ----[relatedto]---> action ----[relatedto*]---> attention ----[relatedto*]---> alarm
slice ----[relatedto]---> action ----[relatedto*]---> disturb ----[relatedto*]---> alarm
slice ----[relatedto]---> action ----[relatedto*]---> ring ----[relatedto*/usedfor*]---> alarm
slice ----[relatedto]---> action ----[relatedto*]---> rouse ----[relatedto*]---> alarm
slice ----[relatedto]---> action ----[relatedto*]---> shock ----[relatedto]---> alarm
slice ----[relatedto]---> action ----[relatedto*]---> smoke ----[relatedto]---> alarm
slice ----[relatedto]---> action ----[relatedto*]---> surprise ----[relatedto*]---> alarm
slice ----[partof]---> orange ----[relatedto*]---> fire ----[relatedto]---> alarm
slice ----[partof]---> orange ----[relatedto]---> warning ----[relatedto*]---> alarm
slice ----[antonym]---> good ----[relatedto*]---> morning ----[relatedto]---> alarm
slice ----[antonym]---> good ----

...

bedroom ----[relatedto]---> alarm
bedroom ----[relatedto*/partof*/relatedto]---> bed ----[relatedto]---> alarm
bedroom ----[relatedto]---> alarm_clock ----[relatedto*/relatedto]---> alarm
bedroom ----[relatedto]---> clock ----[relatedto]---> alarm
bedroom ----[relatedto*/atlocation]---> apartment ----[atlocation*]---> bed ----[relatedto]---> alarm
bedroom ----[relatedto*/atlocation]---> apartment ----[atlocation*]---> smoke_detector ----[relatedto]---> alarm
bedroom ----[relatedto*/atlocation/atlocation*/relatedto]---> house ----[atlocation*]---> fire ----[relatedto]---> alarm
bedroom ----[relatedto*/atlocation/atlocation*/relatedto]---> house ----[relatedto*/atlocation*]---> bed ----[relatedto]---> alarm
bedroom ----[relatedto*/atlocation/atlocation*/relatedto]---> house ----[atlocation*]---> clock ----[relatedto]---> alarm
bedroom ----[relatedto*/atlocation/atlocation*/relatedto]---> house ----[relatedto*]---> bell ----[relatedto]---> alarm
bedroom ----[relatedto*/relatedto]---> slee

 99%|█████████▉| 86/87 [00:53<00:00,  2.60it/s]

bathroom ----[relatedto*]---> bed ----[relatedto]---> alarm
bathroom ----[antonym*/atlocation]---> bedroom ----[relatedto]---> alarm
bathroom ----[atlocation*]---> human ----[atlocation]---> bed ----[relatedto]---> alarm
bathroom ----[atlocation*]---> human ----[relatedto*]---> fear ----[relatedto*/isa*]---> alarm
bathroom ----[atlocation*]---> human ----[relatedto*]---> sound ----[relatedto]---> alarm
bathroom ----[atlocation*]---> water ----[antonym/antonym*/relatedto*/relatedto]---> fire ----[relatedto]---> alarm
bathroom ----[atlocation*]---> water ----[relatedto]---> device ----[relatedto*/isa*]---> alarm
bathroom ----[atlocation]---> apartment ----[atlocation*]---> bed ----[relatedto]---> alarm
bathroom ----[atlocation]---> apartment ----[atlocation*/relatedto]---> bedroom ----[relatedto]---> alarm
bathroom ----[atlocation]---> apartment ----[atlocation*]---> smoke_detector ----[relatedto]---> alarm
bathroom ----[partof/atlocation]---> house ----[atlocation*]---> fire ----[relate

100%|██████████| 87/87 [00:53<00:00,  1.62it/s]

livingroom ----[atlocation]---> house ----[atlocation*]---> fire ----[relatedto]---> alarm
livingroom ----[atlocation]---> house ----[relatedto*/atlocation*]---> bed ----[relatedto]---> alarm
livingroom ----[atlocation]---> house ----[relatedto*/atlocation/atlocation*/relatedto]---> bedroom ----[relatedto]---> alarm
livingroom ----[atlocation]---> house ----[atlocation*]---> clock ----[relatedto]---> alarm
livingroom ----[atlocation]---> house ----[relatedto*]---> bell ----[relatedto]---> alarm
livingroom ----[atlocation]---> home ----[atlocation*]---> alarm_clock ----[relatedto*/relatedto]---> alarm
livingroom ----[atlocation]---> home ----[relatedto]---> bedroom ----[relatedto]---> alarm
livingroom ----[atlocation*]---> couch ----[relatedto*/antonym*/isa/relatedto]---> bed ----[relatedto]---> alarm
livingroom ----[atlocation*]---> sofa ----[relatedto*/antonym*]---> bed ----[relatedto]---> alarm
livingroom ----[atlocation*]---> table ----[relatedto*]---> bed ----[relatedto]---> alarm


In [14]:
pathfinder.find_paths('clock', 'bedroom', 1, ifprint=True)

clock ----[relatedto*]---> bedroom


[{'path': [16024, 2068], 'rel': [[32]]}]

In [46]:
from dgl import DGLGraph

Using backend: pytorch


In [47]:
dg = DGLGraph()
dg.from_networkx(cpnet_simple)

/Users/odoemoo1/anaconda3/envs/kagnet_test/lib/python3.6/site-packages/dgl/base.py:25: UserWarning: multigraph will be deprecated.DGL will treat all graphs as multigraph in the future.
  warnings.warn(msg, warn_type)


In [49]:
cpnet_simple.number_of_nodes()

669941

In [61]:
dg.subgraph([10, 20, 500]).adjacency_matrix()

tensor(indices=tensor([], size=(2, 0)),
       values=tensor([], size=(0,)),
       size=(3, 3), nnz=0, layout=torch.sparse_coo)

TODO:
    - can you get a subgraph with just the nodes of AI2thor objects as concepts, without pathfinding? 